In [ ]:
import ee
import geemap
import hvplot.pandas
import hvplot.xarray
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path
import shapely

In [ ]:
ee.Authenticate()

In [ ]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map

In [ ]:
import json
import urllib.request

def get_mrc_metadata(return_gdf=True, verbose=False):
    """ Get Metadata from Mekong River Comission Data Portal. """
    url = r'https://api.mrcmekong.org/api/v1/ts/inventory/timeSeriesList'
    urllib.request.urlretrieve(url, 'timeSeriesList.json')
    
    if verbose:
        print(f'Downloaded time-series metadata from {url} .')
    
    f = open('timeSeriesList.json')
    data = json.load(f)
    df_metadata = pd.DataFrame([])
    for dataset in data:
        df_temp = pd.DataFrame([dict(dataset)])
        df_temp['longitude'] = df_temp['longitude'].astype(float)
        df_temp['latitude'] = df_temp['latitude'].astype(float)
        df_metadata = pd.concat([df_temp, df_metadata])
        f.close()
    df_metadata = df_metadata.reset_index().drop(columns=['index'])
    
    if verbose:
        print(f'Found a total of {df_metadata.shape[0]} time-series datasets from {len(df_metadata.stationCode.unique())} stations of the MRC Data Portal.')
        
    if return_gdf:
        gdf_metadata = gpd.GeoDataFrame(
            df_metadata, geometry=gpd.points_from_xy(df_metadata.longitude, df_metadata.latitude), crs="EPSG:4326"
        )
        return(gdf_metadata)
    else:
        return(df_metadata)

gdf_metadata = get_mrc_metadata(return_gdf=True)
gdf_metadata_dmsp = gdf_metadata.loc[gdf_metadata.label.str.contains('DSMP')]
gdf_stations_dsmp = gdf_metadata_dmsp.groupby('locationIdentifier').first()[['river', 'stationShortName', 'geometry']].set_crs('EPSG:4326')
Map.add_gdf(gdf_stations_dsmp, 'MRC DSMP stations', {'color': 'red'})

In [ ]:
# Load data from DSMP surves
paths_data_s = list(Path(f'../mrc_webscrapper/outputs/csv/Sediment Concentration/').glob(f'*.csv'))
paths_data_q = list(Path(f'../mrc_webscrapper/outputs/csv/Discharge/').glob(f'*.csv'))
paths_data = paths_data_q + paths_data_s
df_data = pd.DataFrame([])
for path in paths_data_s:
    df_temp = pd.read_csv(path, dtype={'station_code':'str'})
    df_temp['date_utc'] = pd.to_datetime(df_temp['date'])
    df_temp['med_frq'] = np.median(np.diff(df_temp.date_utc))
    df_data = pd.concat([df_data, df_temp])

df_data_dsmp = df_data.loc[df_data.identifier.str.contains('DSMP')]
df_dsmp_stations = df_data_dsmp.groupby('station_code').first()
df_data_dsmp = df_data.loc[df_data.station_code.isin(df_dsmp_stations.index)]
    
gdf_stations = gpd.GeoDataFrame(df_dsmp_stations,
                 crs={'init': 'epsg:4326'},
                 geometry=df_dsmp_stations.apply(lambda row: shapely.geometry.Point((row.lon, row.lat)), axis=1)
                )

gdf_data_dsmp = gpd.GeoDataFrame(df_data_dsmp.join(gdf_stations.geometry, on='station_code'))

In [ ]:
# Set timespan
start_date, end_date = '2000-01-01', '2024-12-31'

# Cloud masking (scene-based)
cld_filt_thresh = 80        # Maximum image cloud cover percent allowed in image collection

# water masking
mask_water = True

# # Cloud masking (pixel-based, s2cloudless only)
# cld_prb_thresh = 25      # Cloud probability (%); values greater than are considered cloud
# cld_prj_dist = 1          # Maximum distance (km) to search for cloud shadows from cloud edges (based on Hollstein decision tree)

# Cloud masking (pixel-based, cloud score+ only)
qa_band = 'cs_cdf'
clear_thresh = 0.75

In [ ]:
# load 3S basin
json_data = 'geometries/geoms.geojson'
fc_geoms = geemap.geojson_to_ee(json_data)
roi_geom = fc_geoms.first().geometry()

# load dams
df = pd.read_csv('geometries/3SReservoirs.csv')
gdf_dams = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_xy(df['X'], df['Y']), crs=4326).drop(columns=['X', 'Y']).set_index('id')
fc_dams = geemap.geopandas_to_ee(gdf_dams)

# define stations
stations = ee.FeatureCollection([
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([106.39220569493087, 14.11961739545873]), {'station_id': '430102', 'station_name': 'Siempang'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([107.78237137912318, 12.897991717109619]), {'station_id': '451305', 'station_name': 'Ban Don'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([106.5278998223061, 13.553039642985812]), {'station_id': '450101', 'station_name': 'Lum Phat'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([107.47041987706022, 13.940093424442852]), {'station_id': '440202', 'station_name': 'Pleiku'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([107.10639453530615, 14.050489408398247]), {'station_id': '440103', 'station_name': 'Andaung Meas'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([107.44760623609672, 13.792268298200513]), {'station_id': '440100', 'station_name': 'Phum Pi'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([105.987503299606, 13.537303972542501]), {'station_id': '014501', 'station_name': 'Stung Treng'}),
])

lake_rhone = ee.Feature(ee.FeatureCollection('users/michaelbrechbuehler/Landsat_ST/shapefiles/rhone_wgs84').first().geometry().centroid(), {'station_id': 'Rhonesee'})
lake_steisee = ee.Feature(ee.FeatureCollection('users/michaelbrechbuehler/Landsat_ST/shapefiles/steisee_wgs84').first().geometry().centroid(), {'station_id': 'Steinsee'})

stations = lake_rhone

# # add S2 clear median
# filters = ee.Filter.And(
#     ee.Filter.bounds(roi_geom), 
#     ee.Filter.date('2017-12-01', '2019-03-31'),
#     ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)
# )
# s2_median = ee.ImageCollection("COPERNICUS/S2_SR").filter(filters).median()

# Map.addLayer(s2_median, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, 'Sentinel-2 RGB')
#Map.addLayer(roi_geom, {'color': 'yellow'}, '3S Basin')
Map.addLayer(stations, {'color': 'red'}, 'Stations')
Map.add_gdf(gdf_dams, 'Dams', {'color': 'blue'})
Map.centerObject(stations)

## Get data

In [ ]:
def process_station(ic_rs, max_diff=1):
    def wrap(fc_station):
        fc_matchups = get_matchups(fc_station, ic_rs, max_diff).map(get_sample)
        return fc_matchups
    return wrap

def run_station_task(ic_rs, sensor, max_diff=1):
    def wrap(fc_station):
        fc_matchups = get_matchups(fc_station, ic_rs, max_diff).map(get_sample)
        task = ee.batch.Export.table.toDrive(**{
            'collection': fc_matchups, 
            'description': f'TSS_export_{sensor}',
            'folder': rf'Earth Engine/TSM/{sensor}'})
        task.start()
        return task
    return wrap

def get_matchups(fc_station, ic_rs, max_diff=1):
    """ Matches FeatureCollection with closest match from ImageCollection. """
    geometry = ee.FeatureCollection(fc_station).geometry()
    ic_rs = ic_rs.filter(ee.Filter.bounds(geometry))
    max_diff_filter = ee.Filter.maxDifference(**{
      'difference': max_diff * 24 * 60 * 60 * 1000,
      'leftField': 'system:time_start',
      'rightField': 'system:time_start'
    })
    save_best_join = ee.Join.saveBest(**{
      'matchKey': 'bestImage',
      'measureKey': 'timeDiff'
    })
    fc_matchups = save_best_join.apply(fc_station, ic_rs, max_diff_filter);
    return fc_matchups

def get_sample(feature):
    """ Sample matched image at feature geometry and add aggregated value as property. """
    feature = ee.Feature(feature)
    match_img = ee.Image(feature.get('bestImage'))
    geometry = feature.geometry().buffer(200)
    value = feature.get('value')
    samples_agg = match_img.reduceRegion(reducer=ee.Reducer.median(), geometry=geometry)
    feature = feature.set('values_eo', samples_agg).set('bestImage', match_img.get('system:index')) 
    return(feature)

In [ ]:
fn

In [ ]:
%load_ext autoreload
%autoreload 2
import functions_process as funcs_process
import functions_turbidity as funcs_turb
import functions_sampling as funcs_sampling
from tqdm.notebook import tqdm

# set timespan
start_date, end_date = '2000-01-01', '2024-12-31'

# prepare insitu data
df = gpd.read_file('input/insitu_data.csv', ignore_geometry=True)
df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry']).set_crs('4326')
gdf_data = gpd.GeoDataFrame(df)
gdf_data = gdf_data.loc[~gdf_data.geometry.is_empty]
#gdf_data = gdf_data.loc[gdf_data.source=='WQMN']

tasks = []
for identifier in tqdm(gdf_data.identifier.unique()):
    fn = identifier.split('@')[1].replace(' ', '').replace('[', '').replace(']', '').replace('(', '').replace(')', '')
    gdf_data_station = gdf_data.loc[gdf_data.identifier==identifier]
    gdf_data_station['date'] = gdf_data_station.date.apply(lambda x: str(x)[:19])
    fc_station = ee.FeatureCollection(geemap.gdf_to_ee(
        gdf_data_station, 
        date='date', date_format='YYYY-MM-dd HH:mm:ss'))
    bounds = fc_station.geometry()
    ic_oli = funcs_process.load_sr_imcoll(sensor='oli', start_date=start_date, end_date=end_date, bounds=bounds, watermask='qa')
    ic_etm = funcs_process.load_sr_imcoll(sensor='etm', start_date=start_date, end_date=end_date, bounds=bounds, watermask='qa')
    ic_msi = funcs_process.load_sr_imcoll(sensor='msi', start_date=start_date, end_date=end_date, bounds=bounds, watermask='scl')
    fc_matchups_oli = get_matchups(fc_station, ic_oli, max_diff=1)
    fc_matchups_msi = get_matchups(fc_station, ic_msi, max_diff=1)
    fc_matchups_etm = get_matchups(fc_station, ic_etm, max_diff=1)
    fc_matchups = ee.FeatureCollection([fc_matchups_oli, fc_matchups_msi, fc_matchups_etm]).flatten().map(get_sample)
    task = ee.batch.Export.table.toDrive(**{
        'collection': fc_matchups, 
        'description': f'TSS_{fn}',
        'folder': 'Earth Engine/TSS'})
    task.start()
    tasks.append(task)

In [ ]:
import time
from datetime import datetime, timedelta

def check_tasks_status(tasks):
    """ Check the state of all provided ee.task objects and posts status updates. """
    colordict = {'white': '\033[0m', 'red': '\033[91m', 'orange': '\033[93m', 'green': '\033[92m'}
    states = []
    for task in tasks:
        # get state and times
        status = task.status()
        state = status['state']
        task_id = status['id']
        time_start, time_update = status['creation_timestamp_ms'], status['update_timestamp_ms']
        time_elapsed = timedelta(milliseconds=(time_update-time_start))
        time_now = datetime.now()
        # set output color
        if state == 'COMPLETED':
            color ='green'
        elif (state == 'RUNNING') | (state == 'READY'):
            color = 'orange'
        elif (state == 'FAILED') | (state == 'CANCEL_REQUESTED') | (state == 'CANCELLED'):
            color = 'red'
        else:
            color = 'white'
        # print msg
        status_msg = f"[{str(time_now)[:19]}] Task {task_id}" \
                     f"({status.get('description', 'No description')}): {colordict[color]+state+colordict['white']}"
                     #f" (runtime: {time_elapsed.seconds/60:0.1f}min)"
        print(status_msg)
        states.append(state)
    return states

# tasks = []
# for sensor, fc in [('oli', fc_matchups_oli), ('etm', fc_matchups_etm), ('msi', fc_matchups_msi)]:
#     task = ee.batch.Export.table.toDrive(**{
#         'collection': fc, 
#         'description': f'TSS_export_{sensor}',
#         'folder': 'Earth Engine'})
#     task.start()
#     tasks.append(task)

all_completed = False
while not all_completed:
    check_tasks_status(tasks)
    states = [task.status()['state'] for task in tasks]
    if all(state in ['COMPLETED', 'FAILED', 'CANCEL_REQUESTED'] for state in states):
        all_completed = True
    else:
        time.sleep(30)

print("All export tasks finished.")

In [ ]:
import time
from tqdm.notebook import tqdm

def monitor_ee_tasks(tasks, check_interval=30):
    """
    Monitor a list of GEE tasks and display tqdm.notebook progress bars for each.

    Parameters:
    - tasks: A list of ee.task objects to monitor.
    - check_interval: Time in seconds between status checks.
    """
    # Initialize a dictionary to hold our progress bars
    progress_bars = {}

    # Initialize progress bars for each task
    for task in tasks:
        status = task.status()
        progress_bars[status['id']] = tqdm(total=100, desc=f"Task {status['description']}")
    try:
        tasks_complete = []
        while len(tasks)!=len(tasks_complete):
            statuses = [task.status() for task in tasks]            
            for status in statuses:
                id = status['id']
                if status['state'] in ['FAILED', 'CANCELLED', 'CANCEL_REQUESTED']:
                        progress_bars[id].bar_style = 'danger'
                elif status['state'] in ['COMPLETED']:
                    progress_bars[id].n = 100
                    progress_bars[id].refresh()
                    progress_bars[id].close()
                    tasks_complete.append(status['id'])
                else:
                    progress = status.get('progress', 0)
                    progress_bars[id].n = progress
                    progress_bars[id].refresh()            
            # Wait for a bit before checking the status again
            time.sleep(check_interval)
    except KeyboardInterrupt:
        print("Monitoring interrupted.")

monitor_ee_tasks(tasks)

In [ ]:
# comput tsm features
# ic_all = ic_all \
#     .map(funcs_turb.calc_spm_nechad) \
#     .map(funcs_turb.calc_tur_nechad) \
#     .map(funcs_turb.calc_tur_dogliotti) \
#     .map(funcs_turb.calc_indices)

In [ ]:
crs = ic_msi.first().select(0).projection().crs()
scale = 30

# wxee convert to xarray
#ds_msi = ic_msi.select('B4').limit(25).wx.to_xarray(scale=scale, region=bounds)

# geemap export
geemap.ee_export_image_collection_to_drive(ee.ImageCollection(ic_msi).select('B4'), folder='export/oli', maxPixels=200000000, region=bounds, scale=30)

In [ ]:
# Export red bands to geotiffs
geemap.ee_export_image_collection_to_drive(ee.ImageCollection(imcoll_etm).select('B3'), folder='export/msi', maxPixels=200000000, region=bounds, scale=30)
geemap.ee_export_image_collection_to_drive(ee.ImageCollection(imcoll_oli).select('B4'), folder='export/oli', maxPixels=200000000, region=bounds, scale=30)
geemap.ee_export_image_collection_to_drive(ee.ImageCollection(imcoll_msi).select('B4'), folder='export/oli', maxPixels=200000000, region=bounds, scale=30)